<a href="https://colab.research.google.com/github/danchaud-vincent/tensorflow-deep-learning/blob/main/exercices-solutions/05_Transfer_Learning_in_TensorFlow_Part_2_Fine_tuning_Exercise_solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 05 - Transfer Learning in TensorFlow Part 2: Fine-tuning Exercices


**Questions:**


1. Use feature-extraction to train a transfer learning model on 10% of the Food Vision data for 10 epochs using `tf.keras.applications.EfficientNetB0` as the base model. Use the `ModelCheckpoint` callback to save the weights to file.
2. Fine-tune the last 20 layers of the base model you trained in 2 for another 10 epochs. How did it go?
3. Fine-tune the last 30 layers of the base model you trained in 2 for another 10 epochs. How dit it go ?
4. Write a function to visualize an image from any dataset (train or test file) and any class (e.g. "steak", "pizza",... etc), visualize it and make a prediction on it using a trained model.